<a href="https://colab.research.google.com/github/profteachkids/CHE2064_Spring2023/blob/main/CHE2064Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
x

array([0.        , 0.20833333, 0.41666667, 0.625     , 0.83333333,
       1.04166667, 1.25      , 1.45833333, 1.66666667, 1.875     ,
       2.08333333, 2.29166667, 2.5       , 2.70833333, 2.91666667,
       3.125     , 3.33333333, 3.54166667, 3.75      , 3.95833333,
       4.16666667, 4.375     , 4.58333333, 4.79166667, 5.        ])

In [41]:
y=0.5*x**2 + 2*x +1

In [42]:
y

array([ 1.        ,  1.43836806,  1.92013889,  2.4453125 ,  3.01388889,
        3.62586806,  4.28125   ,  4.98003472,  5.72222222,  6.5078125 ,
        7.33680556,  8.20920139,  9.125     , 10.08420139, 11.08680556,
       12.1328125 , 13.22222222, 14.35503472, 15.53125   , 16.75086806,
       18.01388889, 19.3203125 , 20.67013889, 22.06336806, 23.5       ])

In [43]:
fig=make_subplots()
fig.add_scatter(x=x, y=y, mode='markers')

In [1]:
import numpy as np 
from plotly.subplots import make_subplots
from scipy.optimize import minimize

In [2]:
x= np.linspace(0,5, 25)

In [3]:
ydata = 0.5*(x**2) + 2.*x + 1.
ydata = np.random.normal(ydata, scale=0.3, )

In [4]:
fig = make_subplots()
fig.add_scatter(x=x,y=ydata,mode='markers')

In [5]:
m,b=np.polyfit(x,ydata,1)

In [6]:
fig.add_scatter(x=x,y=m*x+b,mode='lines')

In [7]:
a,b,c = np.polyfit(x,ydata,2)
fig.add_scatter(x=x,y=a*x**2+b*x+c,mode='lines')

In [8]:
a,b,c

(0.48109589875694647, 2.1563018143815977, 0.8594423289560928)

In [9]:
A=np.c_[x**2, x, np.ones_like(x)]

In [10]:
A.T @ A

array([[3321.17904851,  813.80208333,  212.67361111],
       [ 813.80208333,  212.67361111,   62.5       ],
       [ 212.67361111,   62.5       ,   25.        ]])

In [11]:
np.linalg.solve(A.T @ A, A.T @ ydata)

array([0.4810959 , 2.15630181, 0.85944233])

In [12]:
def sse(v):
    a,b,c = v
    model = a*x**2 + b*x + c
    return np.sum((ydata-model)**2)

In [13]:
minimize(sse, (0., -1000., 0.5))

      fun: 1.343866129562511
 hess_inv: array([[ 0.00493164, -0.02465815,  0.01969222],
       [-0.02465815,  0.13215213, -0.1206148 ],
       [ 0.01969222, -0.1206148 ,  0.15401654]])
      jac: array([ 2.98023224e-08, -1.78813934e-07, -2.98023224e-08])
  message: 'Optimization terminated successfully.'
     nfev: 56
      nit: 8
     njev: 14
   status: 0
  success: True
        x: array([0.48109572, 2.15630267, 0.85944167])

In [14]:
!wget https://raw.githubusercontent.com/profteachkids/CHE2064_Spring2023/main/WaterVaporPressure.csv

--2023-01-31 17:45:53--  https://raw.githubusercontent.com/profteachkids/CHE2064_Spring2023/main/WaterVaporPressure.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1300 (1.3K) [text/plain]
Saving to: ‘WaterVaporPressure.csv’

WaterVaporPressure. 100%[===================>]   1.27K  --.-KB/s    in 0s      

2023-01-31 17:45:53 (63.8 MB/s) - ‘WaterVaporPressure.csv’ saved [1300/1300]



In [15]:
PvapData=np.loadtxt('WaterVaporPressure.csv',delimiter=',')

In [16]:
T, Pexp = PvapData.T
logPexp = np.log10(Pexp)

In [17]:
np.polyfit(1/T, logPexp, 1)

array([-2113.94814059,    10.63549233])

In [18]:
def antoine_sse(v):
    a,b,c = v
    model = 10**(a-b/(T+c))
    return np.sum((model - Pexp)**2)

In [19]:
def antoinelog10_sse(v):
    a,b,c = v
    model = (a-b/(T+c))
    return np.sum((model - logPexp)**2)

In [20]:
np.log10(99000)

4.99563519459755

In [21]:
antoine_abc=minimize(antoine_sse,(10.21,1756.,-36.)).x

In [22]:
antoine_abc

array([  10.5016998 , 2064.92654514,    5.87206213])

In [23]:
antoinelog10_abc=minimize(antoinelog10_sse,(10.64,2114.,0.)).x

In [24]:
def antoine(v):
    a,b,c=v
    return 10**(a-b/(T+c))

In [25]:
fig2=make_subplots()
fig2.add_scatter(x=T, y=Pexp, mode='markers')
fig2.add_scatter(x=T, y=antoine(antoine_abc), mode='lines',name='Pexp')
fig2.add_scatter(x=T, y=antoine(antoinelog10_abc), mode='lines',name='logPexp')
fig2.update_yaxes(type="log")

In [26]:
A=np.c_[T, -logPexp, np.ones_like(T)]

In [33]:
a,c,d=np.linalg.solve(A.T@A, A.T @ (T*logPexp))

In [35]:
b=a*c-d

In [36]:
a,b,c

(10.227991116488935, 1771.955100134423, -34.378925933844585)

In [37]:
antoinelog10_abc

array([  10.20868913, 1756.35431231,  -36.02791716])

In [38]:
antoine_abc

array([  10.5016998 , 2064.92654514,    5.87206213])